In [1]:
from CoRe_Dataloader_ECSG import load_pth_file,load_raw_from_pth_file
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import math
import torchinfo
import time
import numpy as np
import wandb
import datetime
from collections import OrderedDict
# trainds = get_dataset()
# train_dl = DataLoader(trainds,batch_size=6,shuffle = True,)
# test_dl = DataLoader(trainds,batch_size=16*2,shuffle = True,)

In [2]:
train_dl, test_dl = load_pth_file(train_dl_batch_size=8)
raw_train_ds, raw_test_ds = load_raw_from_pth_file()

In [3]:
torch.cuda.empty_cache()

In [4]:
class LDB(nn.Module):
    def __init__(self,inD,outD,dropout) -> None:
        super().__init__()
        inD = int(inD)
        outD = int(outD)
        self.seq = nn.Sequential(
            nn.Linear(inD,outD),
            nn.Dropout1d(p = dropout),
            nn.BatchNorm1d(outD)
        )
    def forward(self,x):
        return self.seq(x)


def conv_formula(d_in,kernel_size,padding = 0,dilation = 1,stride = 1):
    return math.floor(((d_in+(2*padding)-(dilation*(kernel_size-1))-1)/stride)+1)


class CDB(nn.Module):
    def __init__(self,x_in,y_in,in_channels:int,out_channels:int,kernel_size:int,stride = 1,padding = 0,dilation = 1,dropout = .2,) -> None:
        super().__init__()
        self.seq = nn.Sequential( # type: ignore
            nn.Conv2d(in_channels=in_channels,
                      out_channels= out_channels,
                      kernel_size=kernel_size,
                      stride = stride,
                      padding = padding,
                      dilation=dilation
                      ),
            nn.BatchNorm2d(out_channels),
            nn.Dropout2d(dropout),
        )
        self.x_out = conv_formula(x_in,kernel_size=kernel_size,stride=stride,padding=padding,dilation=dilation)
        self.y_out = conv_formula(y_in,kernel_size=kernel_size,stride=stride,padding=padding,dilation=dilation)
        self.out_channels = out_channels
    def forward(self,x):
        return self.seq(x)

def new_CDB(x_in,y_in,in_channels:int = 1,out_channels:int = 1,kernel_size:int = 3,stride = 1,padding = 0,dilation = 1,dropout = .2,device = "cpu"):
    cdb = CDB(x_in,y_in,in_channels,out_channels,kernel_size,stride,padding,dilation,dropout).to(device)
    return cdb,cdb.x_out,cdb.y_out,cdb.out_channels

In [5]:
# problems: needs serious resizing to even work on this computer
class simple_ann_cog(nn.Module):
    def __init__(self,x,y,out_classes,device = "cpu") -> None:
        super().__init__()
        self.resize = Resize((200,200))
        self.norm1 = nn.BatchNorm2d(1).to(device)
        self.flatten = nn.Flatten().to(device)
        self.l1 = LDB(x*y,4096,dropout = .2).to(device)
        self.l2 = LDB(4096,4096/2,.2).to(device)
        self.l3 = LDB(4096/2,out_classes,.2).to(device)
        self.seq = nn.Sequential(self.l1,self.l2,self.l3)
        self.softmax = nn.Softmax(-1).to(device)
    def forward(self,x: torch.Tensor):
        ix = self.resize(x)
        xs = ix.shape
        ix = ix.view((xs[0],1,xs[1],xs[2]))
        ix = self.norm1(ix)
        ix = self.flatten(ix)
        ix = self.seq(ix)
        return self.softmax(ix)

In [6]:
# problems: needs serious resizing to even work on this computer
class double_cnn_ann_cog(nn.Module):
    def __init__(self,x,y,out_classes,device = "cpu") -> None:
        super().__init__()
        self.norm1 = nn.BatchNorm2d(1).to(device)
        self.conv1,xout,yout,out_channels = new_CDB(x_in = 400, y_in = 400,in_channels = 1, out_channels = 8,kernel_size = 7,stride = 3,dropout=0,device=device)
        self.conv2,xout,yout,out_channels = new_CDB(x_in = xout,y_in=yout,in_channels=8,out_channels=16,kernel_size=5,stride=2,dropout=0,device=device)
        self.flatten = nn.Flatten().to(device)
        self.l1 = LDB(xout*yout*out_channels,2048,dropout = 0).to(device)
        self.l2 = LDB(2048,1024,0).to(device)
        self.l3 = LDB(1024,out_classes,0).to(device)
        self.seq = nn.Sequential(self.flatten,self.l1,self.l2,self.l3)
        self.softmax = nn.Softmax(-1).to(device)
        print(xout,yout,out_channels)
    def forward(self,x: torch.Tensor):
        xs = x.shape
        ix = x.view((xs[0],1,xs[1],xs[2]))
        ix = self.norm1(ix)
        ix = self.conv1(ix)
        ix = self.conv2(ix)
        ix = self.seq(ix)
        return self.softmax(ix)
# model = double_cnn_ann_cog(400,400,19,"cuda:0")
# torchinfo.summary(model,(1,400,400))

In [7]:
class multi_cnn_ann_cog(nn.Module):
    def __init__(self,x=400,y=400,out_classes=19,device = "cpu") -> None:
        super().__init__()
        self.name = "multi_cnn_ann_cog"
        self.norm1 = nn.BatchNorm2d(1).to(device)
        self.conv1,xout,yout,out_channels = new_CDB(x_in = 400, y_in = 400,in_channels = 1, out_channels = 12,kernel_size = 7,stride = 2,dropout=0,device=device)
        self.conv2,xout,yout,out_channels = new_CDB(x_in = xout,y_in=yout,in_channels=out_channels,out_channels=24,kernel_size=5,stride=2,dropout=0,device=device)
        self.conv3, xout, yout,out_channels = new_CDB(x_in=xout,y_in=yout,in_channels=out_channels,out_channels=36,kernel_size=5,stride=2,dropout=0,device=device)
        print(xout,yout,out_channels,xout*yout*out_channels)
        self.flatten = nn.Flatten().to(device)
        self.l1 = LDB(xout*yout*out_channels,2048,dropout = 0).to(device)
        self.l2 = LDB(2048,1024,0).to(device)
        self.l3 = LDB(1024,out_classes,0).to(device)
        self.seq = nn.Sequential(self.flatten,self.l1,self.l2,self.l3)
        self.softmax = nn.Softmax(-1).to(device)
        print(xout,yout,out_channels)
    def forward(self,x: torch.Tensor):
        xs = x.shape
        ix = x.view((xs[0],1,xs[1],xs[2]))
        ix = self.norm1(ix)
        ix = self.conv1(ix)
        ix = self.conv2(ix)
        ix = self.conv3(ix)
        ix = self.seq(ix)
        return self.softmax(ix)
model = multi_cnn_ann_cog(400,400,19,"cuda:0")
# torchinfo.summary(model,(1,400,400))

47 47 36 79524
47 47 36


In [8]:
class hyper_cnn_ann_cog(nn.Module):
    def __init__(self, x=400, y=400, out_classes=19, device="cpu") -> None:
        super().__init__()
        self.name = "multi_cnn_ann_cog"
        self.norm1 = nn.BatchNorm2d(1).to(device)
        self.conv1, xout, yout, out_channels = new_CDB(
            x_in=400, y_in=400, in_channels=1, out_channels=8, kernel_size=3, stride=2, dropout=0, device=device)
        self.conv2, xout, yout, out_channels = new_CDB(
            x_in=xout, y_in=yout, in_channels=out_channels, out_channels=16, kernel_size=3, stride=2, dropout=0, device=device)
        self.conv3, xout, yout, out_channels = new_CDB(
            x_in=xout, y_in=yout, in_channels=out_channels, out_channels=32, kernel_size=3, stride=2, dropout=0, device=device)
        self.conv4, xout, yout, out_channels = new_CDB(
            x_in=xout, y_in=yout, in_channels=out_channels, out_channels=64, kernel_size=3, stride=2, dropout=0, device=device)
        self.conv5, xout, yout, out_channels = new_CDB(
            x_in=xout, y_in=yout, in_channels=out_channels, out_channels=128, kernel_size=3, stride=2, dropout=0, device=device)
        self.conv6, xout, yout, out_channels = new_CDB(
            x_in=xout, y_in=yout, in_channels=out_channels, out_channels=256, kernel_size=3, stride=2, dropout=0, device=device)
        print(xout, yout, out_channels, xout*yout*out_channels)
        self.flatten = nn.Flatten().to(device)
        self.l1 = LDB(xout*yout*out_channels, 2048, dropout=0).to(device)
        self.l2 = LDB(2048, 1024, 0).to(device)
        self.l3 = LDB(1024, out_classes, 0).to(device)
        self.seq = nn.Sequential(self.flatten, self.l1, self.l2, self.l3)
        self.softmax = nn.Softmax(-1).to(device)

    def forward(self, x: torch.Tensor):
        xs = x.shape
        print(xs)
        ix = x.view((xs[0], 1, xs[1], xs[2]))
        ix = self.norm1(ix)
        ix = self.conv1(ix)
        ix = self.conv2(ix)
        ix = self.conv3(ix)
        ix = self.conv4(ix)
        ix = self.conv5(ix)
        ix = self.conv6(ix)
        ix = self.seq(ix)
        return self.softmax(ix)


# model = hyper_cnn_ann_cog(400, 400, 19, "cuda:0")
# torchsummary.summary(model, (400, 400))


In [9]:
# problems: needs serious resizing to even work on this computer
class multi_dilated_cnn_ann_cog(nn.Module):
    def __init__(self,x,y,out_classes,device = "cpu") -> None:
        super().__init__()
        self.norm1 = nn.BatchNorm2d(1).to(device)
        xout, yout = 400, 400
        self.conv1,xout,yout,out_channels = new_CDB(x_in = xout,y_in=yout,in_channels = 1, out_channels = 12*3,kernel_size = 7,stride = 3,dropout=.4,dilation = 3, device=device)
        self.conv2,xout,yout,out_channels = new_CDB(x_in = xout,y_in=yout,in_channels=out_channels,out_channels=24*3,kernel_size=7,stride=3,dropout=.4,dilation = 3, device=device)
        self.conv3,xout,yout,out_channels = new_CDB(x_in = xout,y_in=yout,in_channels=out_channels,out_channels=36*3,kernel_size=5,stride=2,dropout=.4,dilation = 2, device=device)
        print(xout,yout,out_channels,xout*yout*out_channels)
        self.flatten = nn.Flatten().to(device)
        self.l1 = LDB(xout*yout*out_channels,2048*2,dropout = .4).to(device)
        self.l2 = LDB(2048*2,1024*2,.4).to(device)
        self.l3 = LDB(1024*2,64,.4).to(device)
        self.l4 = LDB(64,19,.1).to(device)
        self.seq = nn.Sequential(self.flatten,self.l1,self.l2,self.l3,self.l4)
        self.softmax = nn.Softmax(-1).to(device)
        print(xout,yout,out_channels)
    def forward(self,x: torch.Tensor):
        xs = x.shape
        ix = x.view((xs[0],1,xs[1],xs[2]))
        ix = self.norm1(ix)
        ix = self.conv1(ix)
        ix = self.conv2(ix)
        ix = self.conv3(ix)
        ix = self.seq(ix)
        return self.softmax(ix)
# model = multi_dilated_cnn_ann_cog(400,400,19,"cuda:0")
# torchinfo.summary(model,(1,400,400))

In [10]:
class hyper_cnn_ann_cog(nn.Module):
    def __init__(self, x=400, y=400, out_classes=19, device="cpu") -> None:
        super().__init__()
        self.name = "multi_cnn_ann_cog"
        self.norm1 = nn.BatchNorm2d(1).to(device)
        self.conv1, xout, yout, out_channels = new_CDB(
            x_in=400, y_in=400, in_channels=1, out_channels=8, kernel_size=3, stride=2, dropout=0, device=device)
        self.conv2, xout, yout, out_channels = new_CDB(
            x_in=xout, y_in=yout, in_channels=out_channels, out_channels=16, kernel_size=3, stride=2, dropout=0, device=device)
        self.conv3, xout, yout, out_channels = new_CDB(
            x_in=xout, y_in=yout, in_channels=out_channels, out_channels=32, kernel_size=3, stride=2, dropout=0, device=device)
        self.conv3, xout, yout, out_channels = new_CDB(
            x_in=xout, y_in=yout, in_channels=out_channels, out_channels=64, kernel_size=3, stride=2, dropout=0, device=device)
        self.conv3, xout, yout, out_channels = new_CDB(
            x_in=xout, y_in=yout, in_channels=out_channels, out_channels=128, kernel_size=3, stride=2, dropout=0, device=device)
        self.conv3, xout, yout, out_channels = new_CDB(
            x_in=xout, y_in=yout, in_channels=out_channels, out_channels=256, kernel_size=3, stride=2, dropout=0, device=device)
        # print(xout, yout, out_channels, xout*yout*out_channels)
        self.flatten = nn.Flatten().to(device)
        self.l1 = LDB(xout*yout*out_channels, 2048, dropout=0).to(device)
        self.l2 = LDB(2048, 1024, 0).to(device)
        self.l3 = LDB(1024, out_classes, 0).to(device)
        self.seq = nn.Sequential(self.flatten, self.l1, self.l2, self.l3)
        self.softmax = nn.Softmax(-1).to(device)
        print(xout, yout, out_channels)

    def forward(self, x: torch.Tensor):
        xs = x.shape
        ix = x.view((xs[0], 1, xs[1], xs[2]))
        ix = self.norm1(ix)
        ix = self.conv1(ix)
        ix = self.conv2(ix)
        ix = self.conv3(ix)
        ix = self.seq(ix)
        return self.softmax(ix)


# model = hyper_cnn_ann_cog(400, 400, 19, "cuda:0")
# torchinfo.summary(model,(1,400,400))


In [11]:
# torch.onnx.export(model, torch.randn(1,400,400).to("cuda:0"), 'test.onnx', input_names=["features"], output_names=["logits"])

In [12]:
import torchmetrics as metrics
acc = metrics.Accuracy(task="multiclass",num_classes=19).to("cuda:0")
auroc = metrics.AUROC(task = "multiclass",num_classes=19).to("cuda:0")


In [13]:
def new_accuracy(model:torch.nn.Module,dl:DataLoader):
    model.eval()
    raw_output = []
    parameters = []
    with torch.no_grad():
        for batch,(sg,params) in enumerate(dl):
            sg = sg.to("cuda:0").to(torch.float)
            params = params[:,0].to("cuda:0").to(torch.long)
            raw_output.append(model(sg).detach())
            parameters.append(params)
    model.train()
    output = torch.vstack(raw_output)
    parameters = torch.hstack(parameters)
    accuracy = acc(output,parameters)
    auc = auroc(output,parameters)
    return accuracy,auc


In [ ]:
startlr = 3e-4
optimizer = optim.Adam(params=model.parameters(), lr=startlr)
optimizer1 = optim.NAdam(params=model.parameters(), lr=startlr)
step_scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[5, 15, 45, 135], gamma=0.9)
# at the end of 600 epochs, the learning rate is 0.000,002,62
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=5, gamma=0.986)
scheduler_pl = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max', factor=0.7, patience=30, verbose=True)
lossfn = nn.CrossEntropyLoss()


In [ ]:
def train_eval_model(config,adam = True,nadam = False):
    tot_acc,auc = new_accuracy(model=model,dl = test_dl)
    max_acc = -1
    max_auc = -1
    for epoch in range(1,config.epochs+2):
        btime = time.time()
        ldl = len(train_dl)
        for batch,(sg,params) in enumerate(train_dl):
            stime = time.time()
            sg = sg.to("cuda:0").to(torch.float)
            params = params[:,0].to("cuda:0").to(torch.long)
            optimizer.zero_grad()
            outputs = model(sg)
            loss = lossfn(outputs,params)
            loss.backward()
            optimizer.step() if adam else None
            optimizer1.step() if nadam else None
            wandb.log({"loss":loss.item(),"batch_accuracy":acc(outputs,params),"lr":scheduler.get_last_lr()[0],"epoch":epoch})
            print(f"{epoch:5}/{config.epochs:5} // {batch:5}/{ldl:5} | Loss: {loss.item():2.4},batch_accuracy:{acc(outputs,params):3.4}, last_total_accuracy: {tot_acc}, Maximum Accuracy {max_acc} last AUROC {auc} Max AUC {max_auc} lr:{scheduler.get_last_lr()[0]:1.5},Time per Batch: {time.time()-stime:1.2} seconds     ",end = "\r",flush=True)
            torch.cuda.empty_cache()
        scheduler.step()
        step_scheduler.step()
        scheduler_pl.step()
        tot_acc,auc = new_accuracy(model=model,dl = test_dl)
        if(tot_acc > max_acc):
            max_acc = tot_acc
            config.best_model = model.state_dict()
            try:
                torch.save(config.best_model, f"./saved_models/cnns/best_model_state_dict_at_for{config.run_name}_stime_{config.start_time.replace(':', '-')}__acc_{max_acc}__auc_{auc}.pt")
            except:  pass    
            print("\nSAVING MODEL")
        max_auc = max(max_auc,auc)
        print(f"\nEpoch {epoch+1}/{config.epochs} finished. Total accuracy: {tot_acc:3.5} AUROC: {auc} Time per Epoch: {time.time()-btime:1.5}")

        wandb.log({"epoch":epoch,"accuracy":tot_acc,"max_accuracy":max_acc,"lr":scheduler.get_last_lr()[0],"auroc":auc})

In [ ]:
wandb.init(project = "sdisc-2d-2convlayer")
config = wandb.config
config.run_name = wandb.run._run_id
config = wandb.config
config.epochs = 1000
config.inx = 400
config.iny = 400
config.lr = startlr     
config.best_model = OrderedDict()
config.start_time = datetime.datetime.now().isoformat()
config.savename = f"best_model_state_dict_at_for{config.run_name}_stime_{config.start_time.replace(':', '-')}__acc_max_acc__auc_auc.pt"

train_eval_model(wandb.config,nadam=True)

In [ ]:
torch.save(config.best_model,
           f"./saved_models/cnns/best_model_state_dict_at_for{config.run_name}_stime_{config.start_time.replace(':', '-')}_BEST_MODEL.pt")


In [ ]:
train_eval_model(config=config,adam = False, nadam = True)

In [14]:
model = multi_cnn_ann_cog(device="cuda:0").load_state_dict(torch.load(r"C:\Users\aashr\Desktop\research\testing_grounds\saved_models\cnns\best_model_state_dict_at_for6mf1ze6n_stime_2023-03-07T16-04-03.828356__acc_0.8732394576072693__auc_0.8983896970748901.pt"))


47 47 36 79524
47 47 36


RuntimeError: Error(s) in loading state_dict for multi_cnn_ann_cog:
	Missing key(s) in state_dict: "norm1.weight", "norm1.bias", "norm1.running_mean", "norm1.running_var", "conv1.seq.0.weight", "conv1.seq.0.bias", "conv1.seq.1.weight", "conv1.seq.1.bias", "conv1.seq.1.running_mean", "conv1.seq.1.running_var", "conv2.seq.0.weight", "conv2.seq.0.bias", "conv2.seq.1.weight", "conv2.seq.1.bias", "conv2.seq.1.running_mean", "conv2.seq.1.running_var", "conv3.seq.0.weight", "conv3.seq.0.bias", "conv3.seq.1.weight", "conv3.seq.1.bias", "conv3.seq.1.running_mean", "conv3.seq.1.running_var", "l1.seq.0.weight", "l1.seq.0.bias", "l1.seq.2.weight", "l1.seq.2.bias", "l1.seq.2.running_mean", "l1.seq.2.running_var", "l2.seq.0.weight", "l2.seq.0.bias", "l2.seq.2.weight", "l2.seq.2.bias", "l2.seq.2.running_mean", "l2.seq.2.running_var", "l3.seq.0.weight", "l3.seq.0.bias", "l3.seq.2.weight", "l3.seq.2.bias", "l3.seq.2.running_mean", "l3.seq.2.running_var", "seq.1.seq.0.weight", "seq.1.seq.0.bias", "seq.1

In [15]:
import glob

In [20]:
glob.glob(r"C:\Users\aashr\Desktop\research\testing_grounds\saved_models\cnns/*2023-03-03*")


['C:\\Users\\aashr\\Desktop\\research\\testing_grounds\\saved_models\\cnns\\best_model_state_dict_at_forc7kk02nb_stime_2023-03-03T22-19-32.826179__acc_0.7336152195930481__auc_0.6865194439888.pt',
 'C:\\Users\\aashr\\Desktop\\research\\testing_grounds\\saved_models\\cnns\\best_model_state_dict_at_forc7kk02nb_stime_2023-03-03T22-19-32.826179__acc_0.7501056790351868__auc_0.6972088813781738.pt',
 'C:\\Users\\aashr\\Desktop\\research\\testing_grounds\\saved_models\\cnns\\best_model_state_dict_at_forc7kk02nb_stime_2023-03-03T22-19-32.826179__acc_0.75095134973526__auc_0.7015225887298584.pt',
 'C:\\Users\\aashr\\Desktop\\research\\testing_grounds\\saved_models\\cnns\\best_model_state_dict_at_forc7kk02nb_stime_2023-03-03T22-19-32.826179__acc_0.762367844581604__auc_0.7025383114814758.pt',
 'C:\\Users\\aashr\\Desktop\\research\\testing_grounds\\saved_models\\cnns\\best_model_state_dict_at_forc7kk02nb_stime_2023-03-03T22-19-32.826179__acc_0.7674418687820435__auc_0.76778644323349.pt',
 'C:\\Users\\